# 2. Продвинутые операции

In [1]:
import os
os.environ['PYSPARK_PYTHON'] = 'python'
os.environ['HADOOP_USER_NAME'] = 'root'  # Обход проверки пользователя

from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder \
    .appName("Test") \
    .master("local[*]") \
    .config("spark.driver.host", "localhost") \
    .config("spark.executor.memory", "2g") \
    .getOrCreate()

## 2.1 Оконные функции

Оконные функции (Window Functions) в PySpark позволяют выполнять вычисления над группами строк, сохраняя при этом индивидуальность каждой строки.

Основные компоненты оконных функций:
1. Оконная спецификация (WindowSpec) - определяет, какие строки будут включены в рамки окна для каждой строки
2. Оконная функция - функция, которая применяется к данным в рамках окна

**Ранжирование** (rank(), dense_rank(), row_number(), percent_rank())  
**Агрегатные функции** (sum(), avg(), max(), min())  
**Смещение** (lag(), lead())  
**Аналитические функции** (first(), last(), cume_dist(), ntile())  

In [10]:
# Задача: Найти разницу между текущей и предыдущей покупкой для каждого пользователя.

# Создаём DataFrame с покупками
sales = spark.createDataFrame([
    (1, "2023-01-10", 100),
    (1, "2023-01-15", 200),
    (2, "2023-01-12", 50),
    (1, "2023-01-20", 300)
], ["user_id", "date", "amount"])

# Определяем окно
from pyspark.sql.window import Window
window = Window.partitionBy("user_id").orderBy("date")

# Добавляем разницу с предыдущей покупкой
from pyspark.sql.functions import lag, col
sales_with_diff = sales.withColumn("prev_amount", lag("amount").over(window)).withColumn("diff", col("amount") - col("prev_amount"))
sales_with_diff.show()

+-------+----------+------+-----------+----+
|user_id|      date|amount|prev_amount|diff|
+-------+----------+------+-----------+----+
|      1|2023-01-10|   100|       NULL|NULL|
|      1|2023-01-15|   200|        100| 100|
|      1|2023-01-20|   300|        200| 100|
|      2|2023-01-12|    50|       NULL|NULL|
+-------+----------+------+-----------+----+



In [3]:
# Топ-3 товара по категориям

# Данные о товарах
products = spark.createDataFrame([
    (1, "Laptop", "Electronics", 999),
    (2, "Phone", "Electronics", 699),
    (3, "Desk", "Furniture", 200),
    (4, "Chair", "Furniture", 150)
], ["product_id", "name", "category", "price"])

# Окно для ранжирования
window = Window.partitionBy("category").orderBy(col("price").desc())

# Топ-3 в каждой категории
from pyspark.sql.functions import dense_rank
top_products = products.withColumn("rank", dense_rank().over(window)).filter(col("rank") <= 3)
top_products.show()

+----------+------+-----------+-----+----+
|product_id|  name|   category|price|rank|
+----------+------+-----------+-----+----+
|         1|Laptop|Electronics|  999|   1|
|         2| Phone|Electronics|  699|   2|
|         3|  Desk|  Furniture|  200|   1|
|         4| Chair|  Furniture|  150|   2|
+----------+------+-----------+-----+----+



## 2.2 Работа с датами и строками

Обработка дат и времени:
- Парсинг строк в даты: to_date(), to_timestamp()
- Извлечение компонентов: year(), month(), dayofweek()
- Арифметика: datediff(), date_add(), months_between()
- Форматирование: date_format()

In [7]:
# Анализ продаж по времени

spark = SparkSession.builder.appName("Data").getOrCreate()

# Создаём DataFrame с датами
sales_data = [
    (1, "2023-01-15", 150.0),
    (2, "15-02-2023", 200.0),  # Нестандартный формат
    (3, "2023/03/20", 99.99)
]
df = spark.createDataFrame(sales_data, ["order_id", "order_date", "amount"])

# Приводим даты к единому формату (yyyy-MM-dd)
df = df.withColumn("parsed_date", 
      to_date(col("order_date"), "yyyy-MM-dd")) \
      .withColumn("parsed_date", 
      coalesce(col("parsed_date"), 
               to_date(col("order_date"), "dd-MM-yyyy"),
               to_date(col("order_date"), "yyyy/MM/dd")))

# Извлекаем год, месяц и день недели
df = df.withColumn("year", year("parsed_date")) \
       .withColumn("month", month("parsed_date")) \
       .withColumn("day_of_week", dayofweek("parsed_date"))

df.show()

+--------+----------+------+-----------+----+-----+-----------+
|order_id|order_date|amount|parsed_date|year|month|day_of_week|
+--------+----------+------+-----------+----+-----+-----------+
|       1|2023-01-15| 150.0| 2023-01-15|2023|    1|          1|
|       2|15-02-2023| 200.0| 2023-02-15|2023|    2|          4|
|       3|2023/03/20| 99.99| 2023-03-20|2023|    3|          2|
+--------+----------+------+-----------+----+-----+-----------+



In [14]:
# Расчёт дней между заказами
from pyspark.sql.window import Window

window = Window.partitionBy("year").orderBy("parsed_date")
df = df.withColumn("days_since_last_order", 
      datediff(col("parsed_date"), lag("parsed_date").over(window)))

df.show()

+--------+----------+------+-----------+----+-----+-----------+---------------------+
|order_id|order_date|amount|parsed_date|year|month|day_of_week|days_since_last_order|
+--------+----------+------+-----------+----+-----+-----------+---------------------+
|       1|2023-01-15| 150.0| 2023-01-15|2023|    1|          1|                 NULL|
|       2|15-02-2023| 200.0| 2023-02-15|2023|    2|          4|                   31|
|       3|2023/03/20| 99.99| 2023-03-20|2023|    3|          2|                   33|
+--------+----------+------+-----------+----+-----+-----------+---------------------+



Работа со строками:
- Базовые операции: concat(), substring(), trim()
- Регулярные выражения: regexp_extract(), regexp_replace()
- Проверки: startswith(), endswith(), contains()

In [16]:
# Очистка и анализ текста

# Извлечение домена из email
users = spark.createDataFrame([
    (1, "alice@example.com"),
    (2, "bob@gmail.com")
], ["user_id", "email"])

users = users.withColumn("domain", 
      regexp_extract(col("email"), "@(.+)$", 1))

users.show()

+-------+-----------------+-----------+
|user_id|            email|     domain|
+-------+-----------------+-----------+
|      1|alice@example.com|example.com|
|      2|    bob@gmail.com|  gmail.com|
+-------+-----------------+-----------+



In [17]:
# Замена цензурой нецензурных слов
comments = spark.createDataFrame([
    (1, "This is bad!"),
    (2, "Worst product ever")
], ["comment_id", "text"])

bad_words = ["bad", "worst"]
pattern = "|".join(bad_words)
comments = comments.withColumn("clean_text", 
      regexp_replace(col("text"), pattern, "***"))

comments.show()

+----------+------------------+------------------+
|comment_id|              text|        clean_text|
+----------+------------------+------------------+
|         1|      This is bad!|      This is ***!|
|         2|Worst product ever|Worst product ever|
+----------+------------------+------------------+



In [21]:
# Загрузка CSV с заголовком и выводом схемы
df_Titanic = spark.read.csv("data/train.csv", header=True, inferSchema=True)

# Разделяем "Surname, Title Name" и извлекаем фамилию
df_Titanic = df_Titanic.withColumn("surname", 
      split(col("Name"), ",")[0])

# Длина имени
df_Titanic = df_Titanic.withColumn("name_length", 
      length(col("Name")))

df_Titanic.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+---------+-----------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|  surname|name_length|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+---------+-----------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| NULL|       S|   Braund|         23|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|  Cumings|         51|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| NULL|       S|Heikkinen|         22|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S| Futrelle|         44|
|          5|       0|     3|Allen, Mr. W

## 2.3 Оптимизация (партиционирование, кэширование)

Партиционирование — разделение данных на части (партиции) для параллельной обработки. Большие файлы (например, 100+ ГБ CSV/Parquet). Частые фильтры по определённым колонкам (например, по дате).

## 2.4 UDF (пользовательские функции)

## 2.5 Чтение/запись в разных форматах

In [22]:
spark.stop()